In [1]:
import pandas as pd
from collections import Counter
import matplotlib.pyplot as plt
from matplotlib import font_manager
import re
from pathlib import Path
from tqdm import tqdm
import json
import pickle
import numpy as np
import collections
import jieba
from collections import Counter
import sklearn
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics  import f1_score,accuracy_score, confusion_matrix
import jieba 
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import classification_report
from sklearn import metrics



with open('./CRECIL/Final_Data/train.json','rb') as infile:
    train_df = json.loads(infile.read())
    

with open('./CRECIL/Final_Data/train_1.pickle','rb') as infile:
    train_df= pickle.load(infile)
    
with open('./CRECIL/Final_Data/dev.json','rb') as infile:
    dev_df = json.loads(infile.read())
    
# save version with extra field for tokenized/prepped dialogues
with open('./CRECIL/Final_Data/dev_1.pickle','rb') as infile:
    dev_df= pickle.load(infile)
    
with open('./CRECIL/Final_Data/test.json','rb') as infile:
    test_df = json.loads(infile.read())
    
with open('./CRECIL/Final_Data/test_1.pickle','rb') as infile:
    test_df= pickle.load(infile)
    
#data load
with open('./CRECIL/My_home_data/partition1.pickle','rb') as infile:
    partition1 = pickle.load(infile)
    
with open('./CRECIL/rid_to_rel.pickle','rb') as infile:
    rid_to_rel = pickle.load(infile)

with open('./CRECIL/rel_to_rid.pickle','rb') as infile:
    rel_to_rid = pickle.load(infile)
    
    
def get_blank_relations(annotations:list) -> list:
    """
    Take the labels and clear out the gold-standard relations, 
    to be filled with predictions by model
    """
    pred_list =[]
    
    for item in annotations:
        copy = item.copy()
        copy['r'] = []
        copy['rid'] = []
        pred_list.append(copy)
    
    return pred_list


def ch_tokenizer(input_str:str):
    #tokenize sentence and return as list
    tokenized = list(jieba.cut(input_str))
    return tokenized

def get_num_speakers(transcript:list)-> int:
    """
    return number of speakers in scene
    """
    ch_set = set()
    for line in transcript:
        ch_set.add(re.findall('S.*(?=:)',line)[0])
    
    total = len(ch_set)
    return total

def dummy_tokenize(phrase): #pass tokenized text version
    return phrase

class results:
    """
    Make an object of the results so that I can get accurate numbers and confusion 
    matrices while sorting for labels.
    """
    def __init__(self,y_test, y_pred):
        self.y_test = y_test
        self.y_pred = y_pred
        self.labels = sorted(list(set(y_test)))
        self.cm = pd.DataFrame(np.zeros(shape=(len(self.labels),len(self.labels)+1))) #initialize to empty
        self.cm[0]=self.labels
        self.cm.columns = ['pred']+self.labels
        self.accuracy = 0
        self.correct = 0
        self.incorrect = 0
        self.label_metrics = dict()
        
        for lab in self.labels:
            self.label_metrics[lab] = dict()
            self.label_metrics[lab]['F1'] = 0.0
            self.label_metrics[lab]['Recall'] = 0.0
            self.label_metrics[lab]['Accuracy'] = 0.0
            self.label_metrics[lab]['TP'] = 0
            self.label_metrics[lab]['TN'] = 0
            self.label_metrics[lab]['FP'] = 0
            self.label_metrics[lab]['FN'] = 0
            self.label_metrics[lab]['cm'] = pd.DataFrame(np.zeros(shape=(2,3)),\
                                                         columns=['']+['Test P','Test N']) #initialize to empty
            self.label_metrics[lab]['cm'][0] = ['Pred P','Pred N']
        
        for i in range(0,len(y_test)):
            
            #update CM
            self.cm.loc[ self.cm['pred'] == y_pred[i], y_test[i]] +=1
            
            if y_pred[i]==y_test[i]:
                self.correct +=1
                self.label_metrics[y_test[i]]['TP'] +=1
                for lab in self.labels:
                    if lab != y_test[i]:
                        self.label_metrics[lab]['TN'] +=1 #all other incorrect labels were not selected
            else:
                self.incorrect +=1
                self.label_metrics[y_test[i]]['FN'] += 1 #true label was not selected
                self.label_metrics[y_pred[i]]['FP'] +=1 #a wrong label was selected
                
                
        self.accuracy = self.correct/(self.correct+self.incorrect)
        for lab in self.labels:
            
            try:
                self.label_metrics[lab]['Precision'] = self.label_metrics[lab]['TP']/(self.label_metrics[lab]['TP']+self.label_metrics[lab]['FP'])
                self.label_metrics[lab]['Recall'] = self.label_metrics[lab]['TP']/(self.label_metrics[lab]['TP']+self.label_metrics[lab]['FN'])
                self.label_metrics[lab]['F1'] = self.label_metrics[lab]['TP']/(self.label_metrics[lab]['TP']+(0.5)*(self.label_metrics[lab]['FP']+self.label_metrics[lab]['FN']))      
                
            except:
                self.label_metrics[lab]['Precision'] = 0
                self.label_metrics[lab]['Recall'] = 0
                self.label_metrics[lab]['F1'] = 0
    def cm(self):
        """
        show cm
        """
        return(self.cm)
    
    def metric(self):
        
        
        print(f"Overall accuracy = {self.accuracy}")
        
        for lab in self.labels:
            print(f"{lab}: {len([item for item in y_test if item==lab])} total")
            print(f"   F1={self.label_metrics[lab]['F1']}")
            print(f"   Precision={self.label_metrics[lab]['Precision']}")
            print(f"   Recall={self.label_metrics[lab]['Recall']}")

        pass


def jb_tokenize(line:str)-> list:
    return list(jieba.cut((line)))


def retrieve_s_lines(speaker:str,dialog:str)-> list:
    # Retrieve lines spoken by speaker
    lines =[]
    
    for line in dialog:
        party = re.findall('^S\s[0-9]',line)[0]
        phrase = re.findall('(?<=: ).*',line)[0]
        if party==speaker:
            tokenized = ch_tokenizer(phrase)
            lines.extend(tokenized)
    return lines




def retrieve_mentions(ch:str,dialog:str)-> list:
    lines =[]
    
    for line in dialog:
        if bool(re.search(ch,line[1])):
            lines.append(line[1])
    return lines

def get_avg_embeddings(array:list)-> dict:
    """Input array is list of sentences, output is a dictionary with the 
    average embeddings of those sentences.
    """
    
    embeddings = []
    for item in array:
        embeddings.append(ft.get_sentence_vector(item))
    avg_embeddings = np.mean(embeddings, axis=0)
    
    return avg_embeddings

def embeddings_to_features(array:list)-> dict:
    
    features = {}
    for iv,value in enumerate(array):
        features['v{}'.format(iv)]=value
    
    return features



    



In [2]:
all_labels = list(rel_to_rid.keys())

X_train = []
y_train = dict()
for lab in all_labels:
    y_train[lab]= []



for i in tqdm(range(0,len(train_df))): 
    
    #list of tuples containing speakers and tokenized sentences
    dialog = train_df[i][0]
    
    #get relations and labels:
    for j in range(0,len(train_df[i][1])):
        
        rel_triplet = train_df[i][1][j]
        x = rel_triplet['x']
        y = rel_triplet['y']
        r = rel_triplet['r'] #note that this is a list that can have len>1

        # retrieve lines by speakers or entity mentionned:
        if bool(re.search('^S\s[0-9]',x)) and bool(re.search('^S\s[0-9]',y)):
            x_lines = retrieve_s_lines(x,dialog)

            y_lines = retrieve_s_lines(y,dialog)

            combined = x_lines + y_lines
            X_train.append(combined)
            current_rels = []
            
            for rel in list(set(r)):

                current_rels.append(rel)
                
            if len(current_rels)>0:
                for lab in all_labels:
                    if lab in current_rels:
                        y_train[lab].append(1)
                    else:
                        y_train[lab].append(0)

  0%|                                                 | 0/482 [00:00<?, ?it/s]Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.465 seconds.
Prefix dict has been built successfully.
100%|███████████████████████████████████████| 482/482 [00:08<00:00, 53.63it/s]


In [3]:
X_dev = []
y_dev = dict()
for lab in all_labels:
    y_dev[lab]= []



for i in tqdm(range(0,len(dev_df))): 
    
    #list of tuples containing speakers and tokenized sentences
    dialog = dev_df[i][0]
    
    #get relations and labels:
    for j in range(0,len(dev_df[i][1])):
        
        rel_triplet = dev_df[i][1][j]
        x = rel_triplet['x']
        y = rel_triplet['y']
        r = rel_triplet['r'] #note that this is a list that can have len>1

        # retrieve lines by speakers or entity mentionned:
        if bool(re.search('^S\s[0-9]',x)) and bool(re.search('^S\s[0-9]',y)):
            x_lines = retrieve_s_lines(x,dialog)

            y_lines = retrieve_s_lines(y,dialog)

            combined = x_lines + y_lines
            X_dev.append(combined)
            current_rels = []
            
            for rel in list(set(r)):
                current_rels.append(rel)
                
            if len(current_rels)>0:
                for lab in all_labels:
                    if lab in current_rels:
                        y_dev[lab].append(1)
                    else:
                        y_dev[lab].append(0)

100%|███████████████████████████████████████| 116/116 [00:01<00:00, 65.93it/s]


In [4]:
X_test = []
y_test = dict()
for lab in all_labels:
    y_test[lab]= []



for i in tqdm(range(0,len(test_df))): 
    
    #list of tuples containing speakers and tokenized sentences
    dialog = test_df[i][0]
    
    #get relations and labels:
    for j in range(0,len(test_df[i][1])):
        
        rel_triplet = test_df[i][1][j]
        x = rel_triplet['x']
        y = rel_triplet['y']
        r = rel_triplet['r'] #note that this is a list that can have len>1

        # retrieve lines by speakers or entity mentionned:
        if bool(re.search('^S\s[0-9]',x)) and bool(re.search('^S\s[0-9]',y)):
            x_lines = retrieve_s_lines(x,dialog)

            y_lines = retrieve_s_lines(y,dialog)

            combined = x_lines + y_lines
            X_test.append(combined)
            current_rels = []
            
            for rel in list(set(r)):



                    current_rels.append(rel)
                
            if len(current_rels)>0:
                for lab in all_labels:
                    if lab in current_rels:
                        y_test[lab].append(1)
                    else:
                        y_test[lab].append(0)
                    


100%|█████████████████████████████████████████| 71/71 [00:01<00:00, 63.17it/s]


In [2]:
# Get feature word list
with open('full_word_list.pickle','rb') as infile:
    word_list = pickle.load(infile)

In [3]:
with open('chinese_mw.txt','r') as infile:
    mws = infile.read().split('\n')
mws =mws[0:len(mws)-1]

In [4]:
word_list = word_list+ mws
word_list = list(set(word_list))
len(word_list)

300

In [ ]:
# Save feature list for readers
with open('custom_feature_set.txt')

In [12]:

def make_word_counts(text,word_list):
    
    
    output_list = []
    for word in word_list:
        output_list.append(len([item for item in text if item==word]))
        
    output = pd.Series(output_list)
    return output

In [13]:
X_train_features = pd.DataFrame([make_word_counts(x,word_list) for x in X_train])
X_train_features.columns = word_list

X_dev_features = pd.DataFrame([make_word_counts(x,word_list) for x in X_dev])
X_dev_features.columns = word_list

X_test_features = pd.DataFrame([make_word_counts(x,word_list) for x in X_test])
X_test_features.columns = word_list

In [14]:
from sklearn.naive_bayes import GaussianNB

GNB = GaussianNB()

In [15]:
y_dev_true = []
y_dev_pred = []

y_test_true = []
y_test_pred = []

for lab in tqdm(all_labels):
    GNB.fit(X_train_features, y_train[lab])
    y_pred_dev = GNB.predict(X_dev_features)
    y_pred_test = GNB.predict(X_test_features)
    
    y_dev_true.append(y_dev[lab])
    y_dev_pred.append(y_pred_dev)
    
    y_test_true.append(y_test[lab])
    y_test_pred.append(y_pred_test)

100%|█████████████████████████████████████████| 32/32 [00:01<00:00, 30.19it/s]


In [16]:
y_dev_true_df = pd.DataFrame(y_dev_true)
y_dev_true_df = y_dev_true_df.transpose()
y_dev_true_df.columns = all_labels

y_dev_pred_df = pd.DataFrame(y_dev_pred)
y_dev_pred_df = y_dev_pred_df.transpose()
y_dev_pred_df.columns = all_labels

In [ ]:
labels = []
precisions = []
recalls = []
true_labs = []
f1s = []

for item in all_labels:
    labels.append(item)
    precisions.append(sklearn.metrics.precision_score(y_dev_true_df[item], y_dev_pred_df[item], labels=all_labels,average='binary'))
    recalls.append(sklearn.metrics.recall_score(y_dev_true_df[item], y_dev_pred_df[item], labels=all_labels,average='binary'))
    f1s.append(sklearn.metrics.f1_score(y_dev_true_df[item], y_dev_pred_df[item], labels=all_labels,average='binary'))
    true_labs.append(Counter(y_dev_true_df[item])[1])
    

In [20]:
labels.append('Avg')
precisions.append(np.mean(precisions))
recalls.append(np.mean(recalls))
f1s.append(np.mean(f1s))
true_labs.append(np.mean(true_labs))

score_df = pd.DataFrame([labels,precisions,recalls,f1s,true_labs])
score_df = score_df.transpose()
score_df.columns = ['labels','precision','recall','f1','support']


In [22]:
with open('dev_score_df.csv','w') as outfile:
    score_df.to_csv(outfile)

In [ ]:
y_test_true_df = pd.DataFrame(y_test_true)
y_test_true_df = y_test_true_df.transpose()
y_test_true_df.columns = all_labels

y_test_pred_df = pd.DataFrame(y_test_pred)
y_test_pred_df = y_test_pred_df.transpose()
y_test_pred_df.columns = all_labels
    
labels = []
precisions = []
recalls = []
true_labs = []
f1s = []

for item in all_labels:
    labels.append(item)
    precisions.append(sklearn.metrics.precision_score(y_test_true_df[item], y_test_pred_df[item], labels=all_labels,average='binary'))
    recalls.append(sklearn.metrics.recall_score(y_test_true_df[item], y_test_pred_df[item], labels=all_labels,average='binary'))
    f1s.append(sklearn.metrics.f1_score(y_test_true_df[item], y_test_pred_df[item], labels=all_labels,average='binary'))
    true_labs.append(Counter(y_test_true_df[item])[1])
    

In [25]:
labels.append('Avg')
precisions.append(np.mean(precisions))
recalls.append(np.mean(recalls))
f1s.append(np.mean(f1s))
true_labs.append(np.mean(true_labs))

score_df = pd.DataFrame([labels,precisions,recalls,f1s,true_labs])
score_df = score_df.transpose()
score_df.columns = ['labels','precision','recall','f1','support']


,labels,precision,recall,f1,support
0,per:children,0.094474,0.921739,0.171382,115
1,per:spouse,0.055344,0.828571,0.103757,70
2,per:parents,0.094474,0.921739,0.171382,115
3,unanswerable,0.388073,0.952703,0.551499,444
4,per:alternate_name,0.0,0.0,0.0,0
5,per:parents-in-law,0.036517,0.95122,0.070334,41
6,per:grandparents,0.02834,0.903226,0.054956,31
7,per:children-in-law,0.036517,0.95122,0.070334,41
8,per:grandchildren,0.02834,0.903226,0.054956,31
9,per:nurse,0.045073,0.934783,0.086,46


In [26]:
with open('test_score_df.csv','w') as outfile:
    score_df.to_csv(outfile)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
NB= DecisionTreeClassifier()
y_test_all = []
y_pred_all = []

for grp in all_groups:
    DT.fit(X_train_features, y_train[grp]['y'])
    y_pred = DT.predict(X_dev_features)
    
    y_test_all.append(y_dev[grp]['y'])
    y_pred_all.append(y_pred)
    
y_test_all_df = pd.DataFrame(y_test_all)
y_test_all_df = y_test_all_df.transpose()
y_test_all_df.columns = all_groups

y_pred_all_df = pd.DataFrame(y_pred_all)
y_pred_all_df = y_pred_all_df.transpose()
y_pred_all_df.columns = all_groups

with open('ref_files/DT_pred.pickle','wb') as outfile:
    pickle.dump(y_pred_all_df,outfile)
    
labels = []
precisions = []
recalls = []
true_labs = []
f1s = []

for item in all_groups:
    labels.append(item)
    precisions.append(sklearn.metrics.precision_score(y_test_all_df[item], y_pred_all_df[item], labels=all_groups,average='binary'))
    recalls.append(sklearn.metrics.recall_score(y_test_all_df[item], y_pred_all_df[item], labels=all_groups,average='binary'))
    f1s.append(sklearn.metrics.f1_score(y_test_all_df[item], y_pred_all_df[item], labels=all_groups,average='binary'))
    true_labs.append(Counter(y_test_all_df[item])[1])
    
labels.append('Avg')
precisions.append(np.mean(precisions))
recalls.append(np.mean(recalls))
f1s.append(np.mean(f1s))
true_labs.append(np.mean(true_labs))

score_df = pd.DataFrame([labels,precisions,recalls,f1s,true_labs])
score_df = score_df.transpose()
score_df.columns = ['labels','precisions','recalls','f1s','true_labs']

with open('ref_files/DT_score_df.pickle','wb') as outfile:
    pickle.dump(score_df,outfile)

In [26]:
X_train_df = pd.DataFrame(X_train_limited)
X_train_df.columns = ['text']
X_train_features = X_train_df.text.apply(lambda x:make_word_counts(x,word_list))
X_train_features.columns = word_list


X_dev_df = pd.DataFrame(X_dev_limited)
X_dev_df.columns = ['text']
X_dev_features = X_dev_df.text.apply(lambda x:make_word_counts(x,word_list))
X_dev_features.columns = word_list

In [27]:
X_dev_features.shape

(1720, 200)

In [28]:
len(y_dev_m)

7422

In [105]:
from sklearn.tree import DecisionTreeClassifier
DT= DecisionTreeClassifier()
y_test_all = []
y_pred_all = []

for grp in all_groups:
    DT.fit(X_train_features, y_train[grp]['y'])
    y_pred = DT.predict(X_dev_features)
    
    y_test_all.append(y_dev[grp]['y'])
    y_pred_all.append(y_pred)
    
y_test_all_df = pd.DataFrame(y_test_all)
y_test_all_df = y_test_all_df.transpose()
y_test_all_df.columns = all_groups

y_pred_all_df = pd.DataFrame(y_pred_all)
y_pred_all_df = y_pred_all_df.transpose()
y_pred_all_df.columns = all_groups

with open('ref_files/DT_pred.pickle','wb') as outfile:
    pickle.dump(y_pred_all_df,outfile)
    
labels = []
precisions = []
recalls = []
true_labs = []
f1s = []

for item in all_groups:
    labels.append(item)
    precisions.append(sklearn.metrics.precision_score(y_test_all_df[item], y_pred_all_df[item], labels=all_groups,average='binary'))
    recalls.append(sklearn.metrics.recall_score(y_test_all_df[item], y_pred_all_df[item], labels=all_groups,average='binary'))
    f1s.append(sklearn.metrics.f1_score(y_test_all_df[item], y_pred_all_df[item], labels=all_groups,average='binary'))
    true_labs.append(Counter(y_test_all_df[item])[1])
    
labels.append('Avg')
precisions.append(np.mean(precisions))
recalls.append(np.mean(recalls))
f1s.append(np.mean(f1s))
true_labs.append(np.mean(true_labs))

score_df = pd.DataFrame([labels,precisions,recalls,f1s,true_labs])
score_df = score_df.transpose()
score_df.columns = ['labels','precisions','recalls','f1s','true_labs']

with open('ref_files/DT_score_df.pickle','wb') as outfile:
    pickle.dump(score_df,outfile)

In [106]:
score_df

,labels,precisions,recalls,f1s,true_labs
0,other,0.458247,0.471606,0.464831,3293
1,family,0.375685,0.300159,0.333702,2512
2,occupational,0.04955,0.019964,0.028461,551
3,friendship,0.205275,0.139191,0.165894,1286
4,Avg,0.272189,0.23273,0.248222,1910.5


In [39]:
from sklearn.naive_bayes import GaussianNB, CategoricalNB, MultinomialNB, BernoulliNB

MNB = MultinomialNB()

MNB.fit(X_train_features, y_train_limited)
y_pred = MNB.predict(X_dev_features)

In [40]:
accuracy_score(y_dev_limited,y_pred)

0.5075581395348837

In [41]:

cv = CountVectorizer(tokenizer=ch_tokenizer,ngram_range=(1,3),min_df=2,max_df=0.55)

#train_array,
X_train_cv = cv.fit_transform(X_train_limited)

X_train_array = X_train_cv.toarray()

X_dev_cv = cv.transform(X_dev_limited)
X_dev_array = X_dev_cv.toarray()


/home2/elyeb/anaconda3/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [42]:
MNB = MultinomialNB()

MNB.fit(X_train_array, y_train_limited)
y_pred = MNB.predict(X_dev_array)

In [43]:
accuracy_score(y_dev_limited,y_pred)

0.627906976744186

In [37]:

BNB = BernoulliNB()

BNB.fit(X_train_features, y_train_limited)
y_pred = BNB.predict(X_dev_features)

In [33]:
Counter(y_pred)

Counter({'family': 1168, 'friendship': 144, 'other': 402, 'occupational': 6})

In [32]:
Counter(y_dev_limited)

Counter({'family': 854, 'friendship': 202, 'other': 593, 'occupational': 71})

In [18]:
labels = []
precisions = []
recalls = []
true_labs = []
f1s = []

for item in list(set(y_train_m)):
    labels.append(item)
    precisions.append(sklearn.metrics.precision_score(y_test_all_df[item], y_pred_all_df[item], labels=all_groups,average='binary'))
    recalls.append(sklearn.metrics.recall_score(y_test_all_df[item], y_pred_all_df[item], labels=all_groups,average='binary'))
    f1s.append(sklearn.metrics.f1_score(y_test_all_df[item], y_pred_all_df[item], labels=all_groups,average='binary'))
    true_labs.append(Counter(y_test_all_df[item])[1])
    
labels.append('Avg')
precisions.append(np.mean(precisions))
recalls.append(np.mean(recalls))
f1s.append(np.mean(f1s))
true_labs.append(np.mean(true_labs))

score_df = pd.DataFrame([labels,precisions,recalls,f1s,true_labs])
score_df = score_df.transpose()
score_df.columns = ['labels','precisions','recalls','f1s','true_labs']

"""
with open('ref_files/DT_score_df.pickle','wb') as outfile:
    pickle.dump(score_df,outfile)
"""
score_df

Counter({'family': 2507,
         'friendship': 1223,
         'other': 3234,
         'occupational': 458})

In [ ]:

    
labels = []
precisions = []
recalls = []
true_labs = []
f1s = []

for item in all_groups:
    labels.append(item)
    precisions.append(sklearn.metrics.precision_score(y_test_all_df[item], y_pred_all_df[item], labels=all_groups,average='binary'))
    recalls.append(sklearn.metrics.recall_score(y_test_all_df[item], y_pred_all_df[item], labels=all_groups,average='binary'))
    f1s.append(sklearn.metrics.f1_score(y_test_all_df[item], y_pred_all_df[item], labels=all_groups,average='binary'))
    true_labs.append(Counter(y_test_all_df[item])[1])
    
labels.append('Avg')
precisions.append(np.mean(precisions))
recalls.append(np.mean(recalls))
f1s.append(np.mean(f1s))
true_labs.append(np.mean(true_labs))

score_df = pd.DataFrame([labels,precisions,recalls,f1s,true_labs])
score_df = score_df.transpose()
score_df.columns = ['labels','precisions','recalls','f1s','true_labs']

"""
with open('ref_files/DT_score_df.pickle','wb') as outfile:
    pickle.dump(score_df,outfile)
"""
score_df

In [107]:
from sklearn.naive_bayes import GaussianNB, CategoricalNB, MultinomialNB, BernoulliNB

MNB = MultinomialNB()


y_test_all = []
y_pred_all = []
y_pred_probs_all = []

for grp in tqdm(all_groups):
    MNB.fit(X_train_features, y_train[grp]['y'])
    y_pred = MNB.predict(X_dev_features)
    y_pred_probs = MNB.predict_proba(X_dev_features)
    prob_1 = [item[1] for item in y_pred_probs]
    
    y_test_all.append(y_dev[grp]['y'])
    y_pred_all.append(y_pred)
    y_pred_probs_all.append(prob_1)
    
y_test_all_df = pd.DataFrame(y_test_all)
y_test_all_df = y_test_all_df.transpose()
y_test_all_df.columns = all_groups

y_pred_all_df = pd.DataFrame(y_pred_all)
y_pred_all_df = y_pred_all_df.transpose()
y_pred_all_df.columns = all_groups

"""with open('ref_files/DT_pred.pickle','wb') as outfile:
    pickle.dump(y_pred_all_df,outfile)"""
    
labels = []
precisions = []
recalls = []
true_labs = []
f1s = []

for item in all_groups:
    labels.append(item)
    precisions.append(sklearn.metrics.precision_score(y_test_all_df[item], y_pred_all_df[item], labels=all_groups,average='binary'))
    recalls.append(sklearn.metrics.recall_score(y_test_all_df[item], y_pred_all_df[item], labels=all_groups,average='binary'))
    f1s.append(sklearn.metrics.f1_score(y_test_all_df[item], y_pred_all_df[item], labels=all_groups,average='binary'))
    true_labs.append(Counter(y_test_all_df[item])[1])
    
labels.append('Avg')
precisions.append(np.mean(precisions))
recalls.append(np.mean(recalls))
f1s.append(np.mean(f1s))
true_labs.append(np.mean(true_labs))

score_df = pd.DataFrame([labels,precisions,recalls,f1s,true_labs])
score_df = score_df.transpose()
score_df.columns = ['labels','precisions','recalls','f1s','true_labs']

"""
with open('ref_files/DT_score_df.pickle','wb') as outfile:
    pickle.dump(score_df,outfile)
"""
score_df

100%|█████████████████████████████████████████████| 4/4 [00:00<00:00, 28.22it/s]


,labels,precisions,recalls,f1s,true_labs
0,other,0.559701,0.273307,0.367272,3293
1,family,0.413514,0.121815,0.188192,2512
2,occupational,0.144628,0.063521,0.088272,551
3,friendship,0.266003,0.145412,0.188034,1286
4,Avg,0.345961,0.151014,0.207943,1910.5


In [109]:
score_df

,labels,precisions,recalls,f1s,true_labs
0,other,0.448578,0.699362,0.546576,3293
1,family,0.344169,0.960987,0.506823,2512
2,occupational,0.071009,0.833031,0.130862,551
3,friendship,0.173751,0.940902,0.293333,1286
4,Avg,0.259377,0.858571,0.369399,1910.5


In [16]:
len(word_list)

200

In [17]:
X_train_array.shape

(35450, 183)

In [12]:
X_dev_array.shape

(7422, 183)

In [ ]:

cv = CountVectorizer(tokenizer=ch_tokenizer,ngram_range=(1,3),min_df=2,max_df=0.55)

#train_array,
X_train_cv = cv.fit_transform(X_train)

X_train_array = X_train_cv.toarray()

X_dev_cv = cv.transform(X_dev)
X_dev_array = X_dev_cv.toarray()


In [11]:
from sklearn.feature_extraction.text import HashingVectorizer

In [16]:
hv = HashingVectorizer(tokenizer=ch_tokenizer,ngram_range=(1,3)) #,min_df=2,max_df=0.55
X_train_hash = hv.fit_transform(X_train)
X_dev_hash = hv.transform(X_dev)

In [17]:
print(X_train_hash.shape)
print(X_dev_hash.shape)

(35450, 1048576)
(7422, 1048576)


In [18]:
with open('ref_files/X_train_hash.pickle','wb') as outfile:
    pickle.dump(X_train_hash,outfile)

with open('ref_files/X_dev_hash.pickle','wb') as outfile:
    pickle.dump(X_dev_hash,outfile)

In [2]:
# These arrays seem to be large and cause other scripts to fail. Try saving them for easier loading:
with open('ref_files/X_train_array.pickle','wb') as outfile:
    pickle.dump(X_train_array,outfile)

with open('ref_files/X_dev_array.pickle','wb') as outfile:
    pickle.dump(X_dev_array,outfile)

In [ ]:
# These arrays seem to be large and cause other scripts to fail. Try saving them for easier loading:
with open('ref_files/X_train_array.pickle','rb') as infile:
    X_train_array =pickle.load(infile)

with open('ref_files/X_dev_array.pickle','rb') as infile:
    X_dev_array = pickle.load(infile)

In [ ]:
y_pred_probs_all = []

y_pred_probs = GBC.predict_proba(X_dev_array)
    prob_1 = [item[1] for item in y_pred_probs]
    y_pred_probs_all.append(prob_1)

In [ ]:
# These arrays seem to be large and cause other scripts to fail. Try saving them for easier loading:
with open('ref_files/X_train_hash.pickle','rb') as infile:
    X_train_hash =pickle.load(infile)

with open('ref_files/X_dev_hash.pickle','rb') as infile:
    X_dev_hash = pickle.load(infile)

In [9]:
probs = MNB.predict_proba(X_dev_array)

In [16]:
y_pred_probs_all.transpose()

AttributeError: 'list' object has no attribute 'transpose'

In [23]:
y_pred_probs_all_df = pd.DataFrame(y_pred_probs_all).transpose()
y_pred_probs_all_df.columns = all_groups

with open('ref_files/y_pred_probs_all_df.pickle','wb') as outfile:
    pickle.dump(y_pred_probs_all_df,outfile)

In [14]:
np.median(prob_1)

1.231732815625558e-12

In [7]:
MNB.classes_

array([0, 1])

In [24]:
y_test_all_df = pd.DataFrame(y_test_all)
y_test_all_df = y_test_all_df.transpose()
y_test_all_df.columns = all_groups

with open('ref_files/dev_gt.pickle','wb') as outfile:
    pickle.dump(y_test_all_df,outfile)

In [ ]:



    

y_pred_all_df = pd.DataFrame(y_pred_all)
y_pred_all_df = y_pred_all_df.transpose()
y_pred_all_df.columns = all_groups

with open('ref_files/MNB_pred.pickle','wb') as outfile:
    pickle.dump(y_pred_all_df,outfile)
    
labels = []
precisions = []
recalls = []
true_labs = []
f1s = []

for item in all_groups:
    labels.append(item)
    precisions.append(sklearn.metrics.precision_score(y_test_all_df[item], y_pred_all_df[item], labels=all_groups,average='binary'))
    recalls.append(sklearn.metrics.recall_score(y_test_all_df[item], y_pred_all_df[item], labels=all_groups,average='binary'))
    f1s.append(sklearn.metrics.f1_score(y_test_all_df[item], y_pred_all_df[item], labels=all_groups,average='binary'))
    true_labs.append(Counter(y_test_all_df[item])[1])
    
labels.append('Avg')
precisions.append(np.mean(precisions))
recalls.append(np.mean(recalls))
f1s.append(np.mean(f1s))
true_labs.append(np.mean(true_labs))

score_df = pd.DataFrame([labels,precisions,recalls,f1s,true_labs])
score_df = score_df.transpose()
score_df.columns = ['labels','precisions','recalls','f1s','true_labs']

with open('ref_files/MNB_score_df.pickle','wb') as outfile:
    pickle.dump(score_df,outfile)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
#fit
KNN = KNeighborsClassifier()

y_test_all = []
y_pred_all = []

for grp in all_groups:
    KNN.fit(X_train_array, y_train[grp]['y'])
    y_pred = GBC.predict(X_dev_array)
    
    y_test_all.append(y_dev[grp]['y'])
    y_pred_all.append(y_pred)
    
y_test_all_df = pd.DataFrame(y_test_all)
y_test_all_df = y_test_all_df.transpose()
y_test_all_df.columns = all_groups

y_pred_all_df = pd.DataFrame(y_pred_all)
y_pred_all_df = y_pred_all_df.transpose()
y_pred_all_df.columns = all_groups

with open('ref_files/KNN_pred.pickle','wb') as outfile:
    pickle.dump(y_pred_all_df,outfile)
    
labels = []
precisions = []
recalls = []
true_labs = []
f1s = []

for item in all_groups:
    labels.append(item)
    precisions.append(sklearn.metrics.precision_score(y_test_all_df[item], y_pred_all_df[item], labels=all_groups,average='binary'))
    recalls.append(sklearn.metrics.recall_score(y_test_all_df[item], y_pred_all_df[item], labels=all_groups,average='binary'))
    f1s.append(sklearn.metrics.f1_score(y_test_all_df[item], y_pred_all_df[item], labels=all_groups,average='binary'))
    true_labs.append(Counter(y_test_all_df[item])[1])
    
labels.append('Avg')
precisions.append(np.mean(precisions))
recalls.append(np.mean(recalls))
f1s.append(np.mean(f1s))
true_labs.append(np.mean(true_labs))

score_df = pd.DataFrame([labels,precisions,recalls,f1s,true_labs])
score_df = score_df.transpose()
score_df.columns = ['labels','precisions','recalls','f1s','true_labs']

with open('ref_files/KNN_score_df.pickle','wb') as outfile:
    pickle.dump(score_df,outfile)

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
#fit
GBC = GradientBoostingClassifier()


y_test_all = []
y_pred_all = []

for grp in all_groups:
    GBC.fit(X_train_array, y_train[grp]['y'])
    y_pred = GBC.predict(X_dev_array)
    
    y_test_all.append(y_dev[grp]['y'])
    y_pred_all.append(y_pred)
    
y_test_all_df = pd.DataFrame(y_test_all)
y_test_all_df = y_test_all_df.transpose()
y_test_all_df.columns = all_groups

y_pred_all_df = pd.DataFrame(y_pred_all)
y_pred_all_df = y_pred_all_df.transpose()
y_pred_all_df.columns = all_groups

with open('ref_files/GBC_pred.pickle','wb') as outfile:
    pickle.dump(y_pred_all_df,outfile)
    
labels = []
precisions = []
recalls = []
true_labs = []
f1s = []

for item in all_groups:
    labels.append(item)
    precisions.append(sklearn.metrics.precision_score(y_test_all_df[item], y_pred_all_df[item], labels=all_groups,average='binary'))
    recalls.append(sklearn.metrics.recall_score(y_test_all_df[item], y_pred_all_df[item], labels=all_groups,average='binary'))
    f1s.append(sklearn.metrics.f1_score(y_test_all_df[item], y_pred_all_df[item], labels=all_groups,average='binary'))
    true_labs.append(Counter(y_test_all_df[item])[1])
    
labels.append('Avg')
precisions.append(np.mean(precisions))
recalls.append(np.mean(recalls))
f1s.append(np.mean(f1s))
true_labs.append(np.mean(true_labs))

score_df = pd.DataFrame([labels,precisions,recalls,f1s,true_labs])
score_df = score_df.transpose()
score_df.columns = ['labels','precisions','recalls','f1s','true_labs']

with open('ref_files/GBC_score_df.pickle','wb') as outfile:
    pickle.dump(score_df,outfile)

In [ ]:
from sklearn.linear_model import SGDClassifier
#fit
SGD = SGDClassifier()


y_test_all = []
y_pred_all = []

for grp in all_groups:
    SGD.fit(X_train_array, y_train[grp]['y'])
    y_pred = SGD.predict(X_dev_array)
    
    y_test_all.append(y_dev[grp]['y'])
    y_pred_all.append(y_pred)
    
y_test_all_df = pd.DataFrame(y_test_all)
y_test_all_df = y_test_all_df.transpose()
y_test_all_df.columns = all_groups

y_pred_all_df = pd.DataFrame(y_pred_all)
y_pred_all_df = y_pred_all_df.transpose()
y_pred_all_df.columns = all_groups

with open('ref_files/SGD_pred.pickle','wb') as outfile:
    pickle.dump(y_pred_all_df,outfile)
    
labels = []
precisions = []
recalls = []
true_labs = []
f1s = []

for item in all_groups:
    labels.append(item)
    precisions.append(sklearn.metrics.precision_score(y_test_all_df[item], y_pred_all_df[item], labels=all_groups,average='binary'))
    recalls.append(sklearn.metrics.recall_score(y_test_all_df[item], y_pred_all_df[item], labels=all_groups,average='binary'))
    f1s.append(sklearn.metrics.f1_score(y_test_all_df[item], y_pred_all_df[item], labels=all_groups,average='binary'))
    true_labs.append(Counter(y_test_all_df[item])[1])
    
labels.append('Avg')
precisions.append(np.mean(precisions))
recalls.append(np.mean(recalls))
f1s.append(np.mean(f1s))
true_labs.append(np.mean(true_labs))

score_df = pd.DataFrame([labels,precisions,recalls,f1s,true_labs])
score_df = score_df.transpose()
score_df.columns = ['labels','precisions','recalls','f1s','true_labs']

with open('ref_files/SGD_score_df.pickle','wb') as outfile:
    pickle.dump(score_df,outfile)

In [ ]:
from sklearn.svm import SVC
#fit
SVM = SVC()

y_test_all = []
y_pred_all = []

for grp in all_groups:
    SVM.fit(X_train_array, y_train[grp]['y'])
    y_pred = SVM.predict(X_dev_array)
    
    y_test_all.append(y_dev[grp]['y'])
    y_pred_all.append(y_pred)
    
y_test_all_df = pd.DataFrame(y_test_all)
y_test_all_df = y_test_all_df.transpose()
y_test_all_df.columns = all_groups

y_pred_all_df = pd.DataFrame(y_pred_all)
y_pred_all_df = y_pred_all_df.transpose()
y_pred_all_df.columns = all_groups

with open('ref_files/SVM_pred.pickle','wb') as outfile:
    pickle.dump(y_pred_all_df,outfile)
    
labels = []
precisions = []
recalls = []
true_labs = []
f1s = []

for item in all_groups:
    labels.append(item)
    precisions.append(sklearn.metrics.precision_score(y_test_all_df[item], y_pred_all_df[item], labels=all_groups,average='binary'))
    recalls.append(sklearn.metrics.recall_score(y_test_all_df[item], y_pred_all_df[item], labels=all_groups,average='binary'))
    f1s.append(sklearn.metrics.f1_score(y_test_all_df[item], y_pred_all_df[item], labels=all_groups,average='binary'))
    true_labs.append(Counter(y_test_all_df[item])[1])
    
labels.append('Avg')
precisions.append(np.mean(precisions))
recalls.append(np.mean(recalls))
f1s.append(np.mean(f1s))
true_labs.append(np.mean(true_labs))

score_df = pd.DataFrame([labels,precisions,recalls,f1s,true_labs])
score_df = score_df.transpose()
score_df.columns = ['labels','precisions','recalls','f1s','true_labs']

with open('ref_files/SVM_score_df.pickle','wb') as outfile:
    pickle.dump(score_df,outfile)

In [ ]:
from sklearn.linear_model import LogisticRegression
LR= LogisticRegression()

y_test_all = []
y_pred_all = []

for grp in all_groups:
    LR.fit(X_train_array, y_train[grp]['y'])
    y_pred = LR.predict(X_dev_array)
    
    y_test_all.append(y_dev[grp]['y'])
    y_pred_all.append(y_pred)
    
y_test_all_df = pd.DataFrame(y_test_all)
y_test_all_df = y_test_all_df.transpose()
y_test_all_df.columns = all_groups

y_pred_all_df = pd.DataFrame(y_pred_all)
y_pred_all_df = y_pred_all_df.transpose()
y_pred_all_df.columns = all_groups

with open('ref_files/LR_pred.pickle','wb') as outfile:
    pickle.dump(y_pred_all_df,outfile)
    
labels = []
precisions = []
recalls = []
true_labs = []
f1s = []

for item in all_groups:
    labels.append(item)
    precisions.append(sklearn.metrics.precision_score(y_test_all_df[item], y_pred_all_df[item], labels=all_groups,average='binary'))
    recalls.append(sklearn.metrics.recall_score(y_test_all_df[item], y_pred_all_df[item], labels=all_groups,average='binary'))
    f1s.append(sklearn.metrics.f1_score(y_test_all_df[item], y_pred_all_df[item], labels=all_groups,average='binary'))
    true_labs.append(Counter(y_test_all_df[item])[1])
    
labels.append('Avg')
precisions.append(np.mean(precisions))
recalls.append(np.mean(recalls))
f1s.append(np.mean(f1s))
true_labs.append(np.mean(true_labs))

score_df = pd.DataFrame([labels,precisions,recalls,f1s,true_labs])
score_df = score_df.transpose()
score_df.columns = ['labels','precisions','recalls','f1s','true_labs']

with open('ref_files/LR_score_df.pickle','wb') as outfile:
    pickle.dump(score_df,outfile)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
#fit
RFC = RandomForestClassifier()

y_test_all = []
y_pred_all = []

for grp in all_groups:
    RFC.fit(X_train_array, y_train[grp]['y'])
    y_pred = RFC.predict(X_dev_array)
    
    y_test_all.append(y_dev[grp]['y'])
    y_pred_all.append(y_pred)
    
y_test_all_df = pd.DataFrame(y_test_all)
y_test_all_df = y_test_all_df.transpose()
y_test_all_df.columns = all_groups

y_pred_all_df = pd.DataFrame(y_pred_all)
y_pred_all_df = y_pred_all_df.transpose()
y_pred_all_df.columns = all_groups

with open('ref_files/RFC_pred.pickle','wb') as outfile:
    pickle.dump(y_pred_all_df,outfile)
    
labels = []
precisions = []
recalls = []
true_labs = []
f1s = []

for item in all_groups:
    labels.append(item)
    precisions.append(sklearn.metrics.precision_score(y_test_all_df[item], y_pred_all_df[item], labels=all_groups,average='binary'))
    recalls.append(sklearn.metrics.recall_score(y_test_all_df[item], y_pred_all_df[item], labels=all_groups,average='binary'))
    f1s.append(sklearn.metrics.f1_score(y_test_all_df[item], y_pred_all_df[item], labels=all_groups,average='binary'))
    true_labs.append(Counter(y_test_all_df[item])[1])
    
labels.append('Avg')
precisions.append(np.mean(precisions))
recalls.append(np.mean(recalls))
f1s.append(np.mean(f1s))
true_labs.append(np.mean(true_labs))

score_df = pd.DataFrame([labels,precisions,recalls,f1s,true_labs])
score_df = score_df.transpose()
score_df.columns = ['labels','precisions','recalls','f1s','true_labs']

with open('ref_files/RFC_score_df.pickle','wb') as outfile:
    pickle.dump(score_df,outfile)

In [ ]:
from sklearn.naive_bayes import GaussianNB
#fit
NB = GaussianNB()

y_test_all = []
y_pred_all = []

for grp in all_groups:
    NB.fit(X_train_array, y_train[grp]['y'])
    y_pred = NB.predict(X_dev_array)
    
    y_test_all.append(y_dev[grp]['y'])
    y_pred_all.append(y_pred)
    
y_test_all_df = pd.DataFrame(y_test_all)
y_test_all_df = y_test_all_df.transpose()
y_test_all_df.columns = all_groups

y_pred_all_df = pd.DataFrame(y_pred_all)
y_pred_all_df = y_pred_all_df.transpose()
y_pred_all_df.columns = all_groups

with open('ref_files/GaussianNB_pred.pickle','wb') as outfile:
    pickle.dump(y_pred_all_df,outfile)
    
labels = []
precisions = []
recalls = []
true_labs = []
f1s = []

for item in all_groups:
    labels.append(item)
    precisions.append(sklearn.metrics.precision_score(y_test_all_df[item], y_pred_all_df[item], labels=all_groups,average='binary'))
    recalls.append(sklearn.metrics.recall_score(y_test_all_df[item], y_pred_all_df[item], labels=all_groups,average='binary'))
    f1s.append(sklearn.metrics.f1_score(y_test_all_df[item], y_pred_all_df[item], labels=all_groups,average='binary'))
    true_labs.append(Counter(y_test_all_df[item])[1])
    
labels.append('Avg')
precisions.append(np.mean(precisions))
recalls.append(np.mean(recalls))
f1s.append(np.mean(f1s))
true_labs.append(np.mean(true_labs))

score_df = pd.DataFrame([labels,precisions,recalls,f1s,true_labs])
score_df = score_df.transpose()
score_df.columns = ['labels','precisions','recalls','f1s','true_labs']

with open('ref_files/GaussianNB_score_df.pickle','wb') as outfile:
    pickle.dump(score_df,outfile)